# Llama-2-7b-chat-hf quantization

This source is based on Lab 2. LLM Quantization Lab(lecture note)


## Part1. LLM Quantization with GPTQ, AWQ, and NF4

### Package pip install & import (~3m)

In [ ]:
print('Installing packages...')
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets
!pip install torch tqdm
!pip install -U bitsandbytes

Installing packages...
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-p_2hac77
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-p_2hac77
  Resolved https://github.com/huggingface/transformers.git to commit 56ff1e92fd3cb808c82f3fa8e79e236dbbea34d5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10255128 sha256=f85b39bbb07ae5c2843566393bfe2e48f62237c2daff22c06d671a8eaae9c9db
  Stored in directory: /tmp/pip-ephem-wheel-cache-nrmvylja/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Cl

#### 런타임 - 세션 다시 시작

In [ ]:
# Import Libraries
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, BitsAndBytesConfig
import datasets
from datasets import load_dataset
from functools import partial
import gc
import copy

In [ ]:
# Set evaluation
import tqdm
def evaluate(model, tokenizer):
    testenc = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
    testenc = tokenizer("\n\n".join(testenc['text']), return_tensors='pt')

    testenc = testenc.input_ids.to(model.device)
    nsamples = 100
    model = model.eval()

    nlls = []
    for i in tqdm.tqdm(range(nsamples), desc="evaluating..."):
        batch = testenc[:, (i * 2048):((i + 1) * 2048)].to(model.device)
        with torch.no_grad():
            lm_logits = model(batch).logits
        shift_logits = lm_logits[:, :-1, :].contiguous().float()
        shift_labels = testenc[:, (i * 2048):((i + 1) * 2048)][:, 1:]
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        neg_log_likelihood = loss.float() * 2048
        nlls.append(neg_log_likelihood)

    return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))


### Part1-0. FP Model

Let's build and load large language model (Llama-2-7b-chat-hf) on GPU

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Kihun` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Kihun`


In [ ]:
model_path = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "meta-llama/Llama-2-7b-chat-hf"

# 모델을 fp16으로 로드
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
fp_model = AutoModelForCausalLM.from_pretrained(model_path,
                                               torch_dtype=torch.float16,  # fp16로 로드
                                               device_map="auto")

# 파라미터 학습 안 함
for para in fp_model.parameters():
    para.requires_grad = False

# 캐시 사용 안 함
fp_model.config.use_cache = False

# 평가 모드로 전환
fp_model.eval()

# 메모리 정리
gc.collect()
torch.cuda.empty_cache()

# 모델 출력 및 데이터 타입 확인
print(fp_model)
print(f"Parameter data type: {fp_model.dtype}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at meta-llama/Llama-2-7b-chat-hf were not used when initializing LlamaForCausalLM: ['model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [ ]:
fp_model_memory = 0

In [ ]:
# Evaluate the model
fp_model_perplexity = evaluate(fp_model, tokenizer)
print(f"\nFP model perplexity: {fp_model_perplexity:.2f}")
gc.collect()
torch.cuda.empty_cache()

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

evaluating...: 100%|██████████| 100/100 [01:07<00:00,  1.48it/s]



FP model perplexity: 6.82


In [ ]:
fp_model_ppl = 0

In [ ]:
# Import Libraries
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, BitsAndBytesConfig
import datasets
from datasets import load_dataset
from functools import partial
import gc
import copy

### Part1-1 Fake RTN Quantization

#### 런타임 - 세션 다시 시작

In [ ]:
import torch
import copy

# 1. 모델 로드 및 초기화
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

# 특정 레이어의 가중치 가져오기 (예: Self-Attention Query Projection Weight)
original_weight = model.model.layers[0].self_attn.q_proj.weight.detach().to('cpu')
org_w_shape = original_weight.shape
print("Original Weight Shape:", org_w_shape)

# 2. 양자화 설정
bit = 4
group_size = 64
max_int = 2**bit - 1
min_int = 0

assert org_w_shape[-1] % group_size == 0, "Group size가 맞지 않습니다!"

# 3. RTN Quantization: 초기 양자화
# 가중치를 그룹 단위로 reshape
reshaped_weight = original_weight.reshape(-1, group_size)

# 그룹별 최소값과 최대값 계산
group_min = reshaped_weight.min(dim=1, keepdim=True)[0]
group_max = reshaped_weight.max(dim=1, keepdim=True)[0]

# 스케일 계산
scale = (group_max - group_min) / max_int
zero_point = torch.round(-group_min / scale).clamp(min_int, max_int)

# 초기 양자화
quantized_weight = torch.round((reshaped_weight / scale) + zero_point).clamp(min_int, max_int)
fake_weight = ((quantized_weight - zero_point) * scale).reshape(org_w_shape)

# 4. 잔차 보정
residual = original_weight - fake_weight
for _ in range(5):  # 5번 반복적으로 보정
    # 잔차를 양자화하여 반영
    reshaped_residual = residual.reshape(-1, group_size)
    residual_min = reshaped_residual.min(dim=1, keepdim=True)[0]
    residual_max = reshaped_residual.max(dim=1, keepdim=True)[0]
    residual_scale = (residual_max - residual_min) / max_int
    residual_zero_point = torch.round(-residual_min / residual_scale).clamp(min_int, max_int)

    quantized_residual = torch.round((reshaped_residual / residual_scale) + residual_zero_point).clamp(min_int, max_int)
    fake_residual = ((quantized_residual - residual_zero_point) * residual_scale).reshape(org_w_shape)

    # 원래 가중치에 보정값 추가
    fake_weight += fake_residual
    residual = original_weight - fake_weight  # 보정된 가중치로 잔차 갱신

# 5. 최종 양자화된 가중치 저장
final_weight = fake_weight
print("Final Quantized Weight:", final_weight)
print("Final Quantized Weight Shape:", final_weight.shape)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at meta-llama/Llama-2-7b-chat-hf were not used when initializing LlamaForCausalLM: ['model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Original Weight Shape: torch.Size([4096, 4096])
Final Quantized Weight: tensor([[-0.0060, -0.0146, -0.0021,  ...,  0.0042,  0.0018, -0.0035],
        [ 0.0142, -0.0043,  0.0032,  ..., -0.0092, -0.0108,  0.0073],
        [-0.0137,  0.0121,  0.0002,  ...,  0.0061,  0.0181, -0.0030],
        ...,
        [ 0.0018,  0.0093, -0.0006,  ...,  0.0092, -0.0289,  0.0085],
        [ 0.0249,  0.0116,  0.0035,  ..., -0.0322, -0.0165, -0.0111],
        [-0.0136, -0.0067,  0.0016,  ...,  0.0176,  0.0175, -0.0083]])
Final Quantized Weight Shape: torch.Size([4096, 4096])


In [ ]:
def evaluate(model, tokenizer):
    from datasets import load_dataset
    import tqdm
    import torch.nn as nn

    testenc = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
    testenc = tokenizer("\n\n".join(testenc['text']), return_tensors='pt')

    testenc = testenc.input_ids.to(model.device)
    nsamples = 100
    model.eval()

    nlls = []
    for i in tqdm.tqdm(range(nsamples), desc="Evaluating..."):
        batch = testenc[:, (i * 2048):((i + 1) * 2048)].to(model.device)
        with torch.no_grad():
            lm_logits = model(batch).logits
        shift_logits = lm_logits[:, :-1, :].contiguous().float()
        shift_labels = testenc[:, (i * 2048):((i + 1) * 2048)][:, 1:]
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        nlls.append(loss)

    return torch.exp(torch.stack(nlls).mean())


In [ ]:
ppl = evaluate(model, tokenizer)
print(f"Perplexity: {ppl.item()}")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Evaluating...: 100%|██████████| 100/100 [02:37<00:00,  1.57s/it]


Perplexity: 6.824859619140625


### Part1-2 NormalFloat 4bit Quantization


- 런타임 - 세션 다시 시작

In [ ]:
# Import Libraries
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, BitsAndBytesConfig
import datasets
from datasets import load_dataset
from functools import partial
import gc
import copy

In [ ]:
# Set evaluation
def evaluate(model, tokenizer):
    testenc = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
    testenc = tokenizer("\n\n".join(testenc['text']), return_tensors='pt')

    testenc = testenc.input_ids.to(model.device)
    nsamples = 100
    model = model.eval()

    nlls = []
    for i in tqdm.tqdm(range(nsamples), desc="evaluating..."):
        batch = testenc[:, (i * 2048):((i + 1) * 2048)].to(model.device)
        with torch.no_grad():
            lm_logits = model(batch).logits
        shift_logits = lm_logits[:, :-1, :].contiguous().float()
        shift_labels = testenc[:, (i * 2048):((i + 1) * 2048)][:, 1:]
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        neg_log_likelihood = loss.float() * 2048
        nlls.append(neg_log_likelihood)

    return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))

In [ ]:
# Load model in NF4
model_path = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32
)
q_model_nf = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
print(q_model_nf)
gc.collect()
torch.cuda.empty_cache()

NameError: name 'BitsAndBytesConfig' is not defined

In [ ]:
q_model_nf_memory = 0

In [ ]:
# Evaluate the NF4 quantized model
q_model_nf_perplexity = evaluate(q_model_nf, tokenizer)
print(f"\nNF4 model perplexity: {q_model_nf_perplexity:.2f}")
gc.collect()
torch.cuda.empty_cache()

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

evaluating...:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
evaluating...: 100%|██████████| 100/100 [05:38<00:00,  3.39s/it]



NF4 model perplexity: 7.01


In [ ]:
q_model_nf_ppl = 0

Structure of NormalFloat 4bit

In [ ]:
# 모델 레이어의 첫 번째 블록(self-attention)을 올바르게 접근하는 예제
print(q_model_nf.model.layers[0].self_attn.k_proj)
print(q_model_nf.model.layers[0].self_attn.k_proj.weight)
print(q_model_nf.model.layers[0].self_attn.k_proj.weight.shape)

Linear4bit(in_features=4096, out_features=4096, bias=False)
Parameter containing:
Parameter(Params4bit([[ 73],
            [120],
            [136],
            ...,
            [101],
            [104],
            [104]], device='cuda:0', dtype=torch.uint8))
torch.Size([8388608, 1])


In [ ]:
print(q_model_nf.model.decoder.layers[0].self_attn.k_proj.quant_state.absmax)
print(q_model_nf.model.decoder.layers[0].self_attn.k_proj.quant_state.absmax.shape)
# 768 * 768 / 64 = 9216

AttributeError: 'LlamaModel' object has no attribute 'decoder'

In [ ]:
print(q_model_nf.model.decoder.layers[0].self_attn.k_proj.quant_state.blocksize)

64


In [ ]:
# Nonuniform Quantization
print(q_model_nf.model)

LlamaModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
        (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
        (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((4096,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding()
)


### Part1-3. GPTQ INT4 Quantization

Let's quantize Llama-2-7b-chat-hf with GPTQ 4-bit


- 런타임 - 런타임 연결해제 및 삭제

In [ ]:
print('Installing packages...')
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets
!pip install torch tqdm
!pip install autoawq
!pip install -U auto-gptq optimum
!pip install --upgrade torchvision

Installing packages...
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-11ywdahk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-11ywdahk
  Resolved https://github.com/huggingface/transformers.git to commit f5620a76344595dbc7c9cff97bbd1edc1696854d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10214440 sha256=43c9b988de86d1e6e10b3927f2bdc0c4961c2d5479da9fe79e604924f8471220
  Stored in directory: /tmp/pip-ephem-wheel-cache-6_vtrgib/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tok

- 세션 다시 시작

In [ ]:
# Import Libraries
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, BitsAndBytesConfig
import datasets
from datasets import load_dataset
from functools import partial
import gc
import copy

In [ ]:
# 4-bit GPTQ with llama2
model_path = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
quantization_config = GPTQConfig(bits=4, dataset = "c4", tokenizer=tokenizer)
q_model_gptq = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", quantization_config=quantization_config)

for para in q_model_gptq.parameters():
    para.requires_grad = False
q_model_gptq.config.use_cache = False
q_model_gptq.eval()
#print(q_model_gptq)
gc.collect()
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5048: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [ ]:
q_model_gptq.save_pretrained("llama-2-7b-chat-gptq")
tokenizer.save_pretrained("llama-2-7b-chat-gptq")

('llama-2-7b-chat-gptq/tokenizer_config.json',
 'llama-2-7b-chat-gptq/special_tokens_map.json',
 'llama-2-7b-chat-gptq/tokenizer.model',
 'llama-2-7b-chat-gptq/added_tokens.json')


- 런타임 - 세션 다시 시작

In [ ]:
# Import Libraries
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, BitsAndBytesConfig
import datasets
from datasets import load_dataset
from functools import partial
import gc
import copy

In [ ]:
# Set evaluation
def evaluate(model, tokenizer):
    testenc = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
    testenc = tokenizer("\n\n".join(testenc['text']), return_tensors='pt')

    testenc = testenc.input_ids.to(model.device)
    nsamples = 100
    model = model.eval()

    nlls = []
    for i in tqdm.tqdm(range(nsamples), desc="evaluating..."):
        batch = testenc[:, (i * 2048):((i + 1) * 2048)].to(model.device)
        with torch.no_grad():
            lm_logits = model(batch).logits
        shift_logits = lm_logits[:, :-1, :].contiguous().float()
        shift_labels = testenc[:, (i * 2048):((i + 1) * 2048)][:, 1:]
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        neg_log_likelihood = loss.float() * 2048
        nlls.append(neg_log_likelihood)

    return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))

In [ ]:
q_model_gptq = AutoModelForCausalLM.from_pretrained("llama-2-7b-chat-gptq", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("llama-2-7b-chat-gptq", use_fast=False)
print(q_model_gptq)
gc.collect()
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5048: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): LlamaMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


In [ ]:
q_model_gptq_memory = 0

In [ ]:
q_model_gptq_perplexity = evaluate(q_model_gptq, tokenizer)
print(f"\n GPTQ 4-bit model perplexity: {q_model_gptq_perplexity:.2f}")
gc.collect()
torch.cuda.empty_cache()

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

evaluating...: 100%|██████████| 100/100 [00:29<00:00,  3.39it/s]



 GPTQ 4-bit model perplexity: 7.02


In [ ]:
q_model_gptq_ppl = 0

How does quantized model looks like?

In [ ]:
model_layers = q_model_gptq.model.layers  # Llama의 레이어 접근 경로
layer_number = 2  # 원하는 레이어 번호

print(f"Inspect Layer {layer_number} k_proj : {model_layers[layer_number].self_attn.k_proj}")
qweight = model_layers[layer_number].self_attn.k_proj.qweight

# qweight shape 출력
print(f"qweight shape is .. {qweight.shape}")
print(f"qweight dim[0] is {q_model_gptq.config.hidden_size / qweight.shape[0]} times smaller than FP weight.")

# qweight 값 출력
print(f"\nqweight \n{qweight}")


Inspect Layer 2 k_proj : QuantLinear()
qweight shape is .. torch.Size([512, 4096])
qweight dim[0] is 8.0 times smaller than FP weight.

qweight 
tensor([[ 2024261303, -1785358486,  1989512346,  ..., -2074433862,
          2052630615, -1806182280],
        [ 1964878649,  -762013505, -1893173158,  ..., -1134802582,
         -1167628428, -1448637765],
        [  983149004, -1364702904,  1968499177,  ..., -1383700378,
          -947279994, -1772722057],
        ...,
        [-1162398058, -1786218410,  1803835305,  ..., -1700328763,
         -2005424214, -1247250737],
        [ 1781095548, -1715974550, -1770480488,  ...,  1940600932,
         -1734895466,  2027693414],
        [-1238799544, -2020100712, -1703449193,  ...,  1973782248,
          2023392663,  -149170022]], device='cuda:0', dtype=torch.int32)


In [ ]:
# Let's check packed weight!
def binary(x, bits):
    mask = 2**torch.arange(bits).to(x.device, x.dtype)
    return x.unsqueeze(-1).bitwise_and(mask).ne(0).byte()

def int_to_dec(x):
    temp = 0
    for i in range(len(x)):
        temp += 2**i * x[-i-1]
    return temp

print(f"Original qweight: {qweight[0,0]}")
print()
print("Unpacked 8 parameters")
for i in range(8):
    unpacked = binary(qweight[0,0], 32)[4*i:4*(i+1)].cpu().numpy()
    print(f"{str(unpacked)} {int_to_dec(unpacked)}")

Original qweight: 2024261303

Unpacked 8 parameters
[1 1 1 0] 14
[1 1 0 1] 13
[0 1 1 0] 6
[0 0 1 1] 3
[1 1 1 0] 14
[0 1 0 1] 5
[0 0 0 1] 1
[1 1 1 0] 14


In [ ]:
# Let's check quantizaion scale!
print(f"scale is {model_layers[layer_number].self_attn.k_proj.scales}")
print(f"\nscale dimension {model_layers[layer_number].self_attn.k_proj.scales.shape}")
print(f"scale dim[0] is {q_model_gptq.config.hidden_size / model_layers[layer_number].self_attn.k_proj.scales.shape[0]} times smaller than original weight")

scale is tensor([[0.0033, 0.0067, 0.0089,  ..., 0.0168, 0.0035, 0.0152],
        [0.0037, 0.0082, 0.0088,  ..., 0.0146, 0.0041, 0.0147],
        [0.0065, 0.0098, 0.0105,  ..., 0.0140, 0.0100, 0.0163],
        ...,
        [0.0087, 0.0081, 0.0086,  ..., 0.0147, 0.0038, 0.0209],
        [0.0054, 0.0084, 0.0146,  ..., 0.0140, 0.0056, 0.0125],
        [0.0036, 0.0074, 0.0081,  ..., 0.0135, 0.0101, 0.0120]],
       device='cuda:0', dtype=torch.float16)

scale dimension torch.Size([32, 4096])
scale dim[0] is 128.0 times smaller than original weight


In [ ]:
# Let's check quantizaion zeros!
print(f"qzeros is {model_layers[layer_number].self_attn.k_proj.qzeros[:2, :10]}")
print(f"\nqzeros dimension {model_layers[layer_number].self_attn.k_proj.qzeros.shape}")
print(f"qzeros dim[0] is {q_model_gptq.config.hidden_size / model_layers[layer_number].self_attn.k_proj.qzeros.shape[0]} times smaller than original weight")
print(f"qzeros dim[1] is {q_model_gptq.config.hidden_size / model_layers[layer_number].self_attn.k_proj.qzeros.shape[1]} times smaller than original weight")

qzeros is tensor([[2004318071, 2004318071, 2004318071, 2004318071, 2004318071, 2004318071,
         2004318071, 2004318071, 2004318071, 2004318071],
        [2004318071, 2004318071, 2004318071, 2004318071, 2004318071, 2004318071,
         2004318071, 2004318071, 2004318071, 2004318071]], device='cuda:0',
       dtype=torch.int32)

qzeros dimension torch.Size([32, 512])
qzeros dim[0] is 128.0 times smaller than original weight
qzeros dim[1] is 8.0 times smaller than original weight


In [ ]:
# Let's check packed zeros!
qzeros = model_layers[layer_number].self_attn.k_proj.qzeros[0,0]

print(qzeros)
print()
for i in range(8):
   unpacked = binary(qzeros, 32)[4*i:4*(i+1)].cpu().numpy()
   print(f"{str(unpacked)} {int_to_dec(unpacked)}")

tensor(2004318071, device='cuda:0', dtype=torch.int32)

[1 1 1 0] 14
[1 1 1 0] 14
[1 1 1 0] 14
[1 1 1 0] 14
[1 1 1 0] 14
[1 1 1 0] 14
[1 1 1 0] 14
[1 1 1 0] 14


### Part1-4. AWQ INT4 Quantization


- 런타임 - 세션 다시 시작

In [ ]:
# Import Libraries
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, BitsAndBytesConfig
import datasets
from datasets import load_dataset
from functools import partial
import gc
import copy


In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = "meta-llama/Llama-2-7b-chat-hf"
#quant_path = 'opt-125m-awq'
quant_config = { "zero_point": True, "q_group_size": 128 }

# Load model
q_model_awq = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=False)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

# for para in q_model_awq.parameters():
#     para.requires_grad = False
# q_model_awq.config.use_cache = False
# q_model_awq.eval()
# gc.collect()
# torch.cuda.empty_cache()

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

LICENSE.txt:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at /root/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf/snapshots/f5db02db724555f92da89c216ac04704f23d4590 were not used when initializing LlamaForCausalLM: ['model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.la

In [ ]:
# Self-Attention 모듈의 k_proj 확인
k_proj_weight = q_model_awq.model.model.layers[0].self_attn.k_proj.weight
print(k_proj_weight)
print(k_proj_weight.shape)
print(k_proj_weight.dtype)



Parameter containing:
tensor([[-0.0155,  0.0078, -0.0011,  ...,  0.0164, -0.0097, -0.0136],
        [ 0.0182,  0.0012,  0.0034,  ..., -0.0206,  0.0143,  0.0229],
        [-0.0245, -0.0220,  0.0018,  ...,  0.0150, -0.0157, -0.0110],
        ...,
        [ 0.0123, -0.0007, -0.0008,  ...,  0.0002,  0.0029,  0.0081],
        [-0.0050,  0.0171, -0.0031,  ..., -0.0033,  0.0112, -0.0110],
        [ 0.0036, -0.0023,  0.0012,  ...,  0.0073, -0.0114,  0.0095]],
       dtype=torch.float16, requires_grad=True)
torch.Size([4096, 4096])
torch.float16


Manually conduct AWQ

In [ ]:
test_awq_model = copy.deepcopy(q_model_awq)
for para in test_awq_model.parameters():
     para.requires_grad = False

from awq.quantize.quantizer import AwqQuantizer
test_awq_model.quantizer = AwqQuantizer(
    test_awq_model,
    test_awq_model.model,
    tokenizer,
    test_awq_model.quant_config.w_bit,
    test_awq_model.quant_config.q_group_size,
    test_awq_model.quant_config.zero_point,
    test_awq_model.quant_config.version,
    calib_data="pileval",
    split="train",
    text_column="text",
    duo_scaling=True,
    modules_to_not_convert=test_awq_model.quant_config.modules_to_not_convert,
    export_compatible=False,
    apply_clip=True,
    n_parallel_calib_samples=None,
    max_calib_samples=128,
    max_calib_seq_len=512,
    max_chunk_memory=1024 * 1024 * 1024,
)

README.md:   0%|          | 0.00/167 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


val.jsonl.zst:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/214670 [00:00<?, ? examples/s]

In [ ]:
print(f"model:  \n {test_awq_model.quantizer.modules}")
print(f"\n inps shape: {test_awq_model.quantizer.inps.shape}")

model:  
 ModuleList(
  (0-31): 32 x LlamaDecoderLayer(
    (self_attn): LlamaSdpaAttention(
      (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
  )
)

 inps shape: torch.Size([65, 512, 4096])


In [ ]:
test_layer = test_awq_model.quantizer.modules[0].to("cuda")
#test_attention_mask = test_awq_model.quantizer.module_kwargs.get("attention_mask").to("cuda")
test_inps = test_awq_model.quantizer.inps.to("cuda")
print(test_layer)

LlamaDecoderLayer(
  (self_attn): LlamaSdpaAttention(
    (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
    (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
    (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
)


In [ ]:
from awq.utils.module import get_named_linears
named_linears = get_named_linears(test_layer)
named_linears

{'self_attn.q_proj': Linear(in_features=4096, out_features=4096, bias=False),
 'self_attn.k_proj': Linear(in_features=4096, out_features=4096, bias=False),
 'self_attn.v_proj': Linear(in_features=4096, out_features=4096, bias=False),
 'self_attn.o_proj': Linear(in_features=4096, out_features=4096, bias=False),
 'mlp.gate_proj': Linear(in_features=4096, out_features=11008, bias=False),
 'mlp.up_proj': Linear(in_features=4096, out_features=11008, bias=False),
 'mlp.down_proj': Linear(in_features=11008, out_features=4096, bias=False)}

In [ ]:
input_feat = test_awq_model.quantizer._get_input_feat(test_layer, named_linears)

# 출력되는 shape
print(input_feat["self_attn.q_proj"].shape)
print(input_feat["self_attn.k_proj"].shape)
print(input_feat["self_attn.v_proj"].shape)

# 'self_attn.out_proj' 키가 없을 경우 기본값 출력
print(input_feat.get("self_attn.out_proj", "Key not found"))

# 'fc1'과 'fc2' 키가 없을 경우 기본값 처리
print(input_feat.get("fc1", "Key 'fc1' not found"))
print(input_feat.get("fc2", "Key 'fc2' not found"))


torch.Size([65, 512, 4096])
torch.Size([65, 512, 4096])
torch.Size([65, 512, 4096])
Key not found
Key 'fc1' not found
Key 'fc2' not found


In [ ]:
module_config = test_awq_model.quantizer.awq_model.get_layers_for_scaling(
    test_layer, input_feat, test_awq_model.quantizer.module_kwargs
)

In [ ]:
#module_config[0]
print(f"prev_op: {module_config[0]['prev_op']}")
print(f"layers: {module_config[0]['layers']}")
print(f"inps: {module_config[0]['inp'].shape}")
print(f"module2inspect: {module_config[0]['module2inspect']}")

prev_op: LlamaRMSNorm((4096,), eps=1e-05)
layers: [Linear(in_features=4096, out_features=4096, bias=False), Linear(in_features=4096, out_features=4096, bias=False), Linear(in_features=4096, out_features=4096, bias=False)]
inps: torch.Size([65, 512, 4096])
module2inspect: LlamaSdpaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
)


In [ ]:
#module_config[1]
print(f"prev_op: {module_config[1]['prev_op']}")
print(f"layers: {module_config[1]['layers']}")
print(f"inps: {module_config[1]['inp'].shape}")

prev_op: Linear(in_features=4096, out_features=4096, bias=False)
layers: [Linear(in_features=4096, out_features=4096, bias=False)]
inps: torch.Size([65, 512, 4096])


In [ ]:
#module_config[2]
print(f"prev_op: {module_config[2]['prev_op']}")
print(f"layers: {module_config[2]['layers']}")
print(f"inps: {module_config[2]['inp'].shape}")

prev_op: LlamaRMSNorm((4096,), eps=1e-05)
layers: [Linear(in_features=4096, out_features=11008, bias=False), Linear(in_features=4096, out_features=11008, bias=False)]
inps: torch.Size([65, 512, 4096])


In [ ]:
#module_config[3]
print(f"prev_op: {module_config[3]['prev_op']}")
print(f"layers: {module_config[3]['layers']}")
print(f"inps: {module_config[3]['inp'].shape}")

prev_op: Linear(in_features=4096, out_features=11008, bias=False)
layers: [Linear(in_features=11008, out_features=4096, bias=False)]
inps: torch.Size([65, 512, 11008])


In [ ]:
scales_list = [
    test_awq_model.quantizer._search_best_scale(test_layer, **layer)
    for layer in module_config
]

In [ ]:
print(scales_list[0][0])
print(scales_list[0][1])
print(scales_list[0][2].shape)

input_layernorm
('self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj')
torch.Size([4096])


In [ ]:
print(scales_list[1][0])
print(scales_list[1][1])
print(scales_list[1][2].shape)

self_attn.v_proj
('self_attn.o_proj',)
torch.Size([4096])


In [ ]:
print(scales_list[2][0])
print(scales_list[2][1])
print(scales_list[2][2].shape)

post_attention_layernorm
('mlp.gate_proj', 'mlp.up_proj')
torch.Size([4096])


In [ ]:
print(scales_list[3][0])
print(scales_list[3][1])
print(scales_list[3][2].shape)

mlp.up_proj
('mlp.down_proj',)
torch.Size([11008])


In [ ]:
# input_layernorm의 weight와 bias 확인
print(f"input_layernorm before scaling: \n {test_layer.input_layernorm.weight[:64]}")

# post_attention_layernorm의 weight와 bias 확인
print(f"post_attention_layernorm before scaling: \n {test_layer.post_attention_layernorm.weight[:64]}")


input_layernorm before scaling: 
 tensor([0.0298, 0.0140, 0.0031, 0.0131, 0.0151, 0.0094, 0.0084, 0.0082, 0.0182,
        0.0220, 0.0052, 0.0046, 0.0043, 0.0203, 0.0135, 0.0088, 0.0067, 0.0103,
        0.0078, 0.0076, 0.0020, 0.0172, 0.4551, 0.0096, 0.0020, 0.0067, 0.0106,
        0.0233, 0.0250, 0.0076, 0.0131, 0.0095, 0.0117, 0.0127, 0.0080, 0.0165,
        0.0061, 0.0145, 0.0072, 0.0130, 0.0059, 0.0153, 0.0094, 0.0162, 0.0059,
        0.0099, 0.0064, 0.0084, 0.0079, 0.0126, 0.0160, 0.0064, 0.0130, 0.0157,
        0.0084, 0.0112, 0.0156, 0.0317, 0.0139, 0.0120, 0.0123, 0.0079, 0.0107,
        0.0141], device='cuda:0', dtype=torch.float16)
post_attention_layernorm before scaling: 
 tensor([0.0508, 0.0525, 0.0498, 0.0457, 0.0483, 0.0532, 0.0557, 0.0481, 0.0537,
        0.0605, 0.0483, 0.0527, 0.0571, 0.0454, 0.0605, 0.0486, 0.0588, 0.0564,
        0.0591, 0.0503, 0.0503, 0.0491, 0.2002, 0.0588, 0.0513, 0.0669, 0.0442,
        0.0669, 0.0518, 0.0515, 0.0510, 0.0540, 0.0547, 0.0559, 0.05

In [ ]:
from awq.quantize.scale import scale_ln_fcs

scale_ln_fcs(
    test_layer.input_layernorm,  # input_layernorm 사용
    [test_layer.mlp.gate_proj],  # 예시로 gate_proj 사용 (다른 fc 계층도 가능)
    scales_list[2][2]
)

In [ ]:
print(f"gate_proj after scaling: \n {test_layer.mlp.gate_proj.weight[:3]}")  # 예시로 gate_proj 사용
print(f"input_layernorm after scaling: \n {test_layer.input_layernorm.weight[:64]}")


gate_proj after scaling: 
 tensor([[ 0.0096,  0.0095,  0.0161,  ..., -0.0081,  0.0039,  0.0073],
        [-0.0015, -0.0033,  0.0032,  ...,  0.0082, -0.0060,  0.0041],
        [ 0.0031, -0.0116,  0.0115,  ..., -0.0064,  0.0125,  0.0102]],
       device='cuda:0', dtype=torch.float16)
input_layernorm after scaling: 
 tensor([0.0537, 0.0254, 0.0055, 0.0238, 0.0274, 0.0165, 0.0153, 0.0150, 0.0330,
        0.0395, 0.0095, 0.0085, 0.0077, 0.0370, 0.0244, 0.0160, 0.0119, 0.0183,
        0.0141, 0.0137, 0.0036, 0.0314, 0.6714, 0.0174, 0.0036, 0.0119, 0.0190,
        0.0416, 0.0448, 0.0135, 0.0236, 0.0172, 0.0214, 0.0231, 0.0145, 0.0299,
        0.0110, 0.0260, 0.0130, 0.0237, 0.0106, 0.0277, 0.0170, 0.0294, 0.0106,
        0.0181, 0.0114, 0.0151, 0.0144, 0.0229, 0.0287, 0.0117, 0.0238, 0.0288,
        0.0153, 0.0204, 0.0283, 0.0579, 0.0252, 0.0220, 0.0223, 0.0145, 0.0196,
        0.0256], device='cuda:0', dtype=torch.float16)


In [ ]:
print(f"Scale values:\n{scales_list[2][2][:64]}")

Scale values:
tensor([0.5547, 0.5532, 0.5615, 0.5522, 0.5522, 0.5698, 0.5464, 0.5449, 0.5513,
        0.5562, 0.5532, 0.5454, 0.5532, 0.5483, 0.5537, 0.5493, 0.5601, 0.5625,
        0.5532, 0.5532, 0.5498, 0.5479, 0.6777, 0.5503, 0.5596, 0.5605, 0.5576,
        0.5610, 0.5586, 0.5586, 0.5571, 0.5532, 0.5454, 0.5503, 0.5513, 0.5513,
        0.5591, 0.5591, 0.5562, 0.5488, 0.5532, 0.5532, 0.5542, 0.5513, 0.5547,
        0.5479, 0.5630, 0.5532, 0.5498, 0.5508, 0.5581, 0.5469, 0.5454, 0.5464,
        0.5479, 0.5479, 0.5522, 0.5483, 0.5513, 0.5444, 0.5508, 0.5474, 0.5493,
        0.5508], dtype=torch.float16)


In [ ]:
# 'gate_proj'의 weight를 저장
before_gate_proj = copy.deepcopy(test_layer.mlp.gate_proj.weight)

# 'gate_proj' weight 비율 출력
print(f"gate_proj weight ratio:\n{(test_layer.mlp.gate_proj.weight / before_gate_proj)[:10]}")


gate_proj weight ratio:
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0', dtype=torch.float16)


In [ ]:
# 'input_layernorm'의 weight 저장
before_ln_weight = copy.deepcopy(test_layer.input_layernorm.weight)

# 'input_layernorm' weight 비율 출력
print(f"input_layernorm weight ratio:\n{(before_ln_weight / test_layer.input_layernorm.weight)[:64]}")


input_layernorm weight ratio:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       dtype=torch.float16)


In [ ]:
# Quantize
q_model_awq.quantize(tokenizer, quant_config=quant_config)
# q_model_awq = q_model_awq.to("cuda:0")
# print(q_model_awq)
# gc.collect()
# torch.cuda.empty_cache()

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 32/32 [29:33<00:00, 55.42s/it]


In [ ]:
# q_model_awq = q_model_awq.to("cpu")
q_model_awq.save_quantized('meta-llama/Llama-2-7b-chat-hf-awq')
tokenizer.save_pretrained('meta-llama/Llama-2-7b-chat-hf-awq')

('meta-llama/Llama-2-7b-chat-hf-awq/tokenizer_config.json',
 'meta-llama/Llama-2-7b-chat-hf-awq/special_tokens_map.json',
 'meta-llama/Llama-2-7b-chat-hf-awq/tokenizer.model',
 'meta-llama/Llama-2-7b-chat-hf-awq/added_tokens.json')

In [ ]:
q_model_awq

LlamaAWQForCausalLM(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
            (k_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
            (v_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
            (o_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
          )
          (mlp): LlamaMLP(
            (gate_proj): WQLinear_GEMM(in_features=4096, out_features=11008, bias=False, w_bit=4, group_size=128)
            (up_proj): WQLinear_GEMM(in_features=4096, out_features=11008, bias=False, w_bit=4, group_size=128)
            (down_proj): WQLinear_GEMM(in_features=11008, out_fe


- 런타임 - 세션 다시 시작

In [ ]:
# Import Libraries
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, BitsAndBytesConfig
import datasets
from datasets import load_dataset
from functools import partial
import gc
import copy

In [ ]:
# Set evaluation
def evaluate(model, tokenizer):
    testenc = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
    testenc = tokenizer("\n\n".join(testenc['text']), return_tensors='pt')

    testenc = testenc.input_ids.to(model.device)
    nsamples = 100
    model = model.eval()

    nlls = []
    for i in tqdm.tqdm(range(nsamples), desc="evaluating..."):
        batch = testenc[:, (i * 2048):((i + 1) * 2048)].to(model.device)
        with torch.no_grad():
            lm_logits = model(batch).logits
        shift_logits = lm_logits[:, :-1, :].contiguous().float()
        shift_labels = testenc[:, (i * 2048):((i + 1) * 2048)][:, 1:]
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        neg_log_likelihood = loss.float() * 2048
        nlls.append(neg_log_likelihood)

    return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))

In [ ]:
q_model_awq = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-chat-hf-awq', device_map="auto")
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf-awq', use_fast=False)
print(q_model_awq)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
          (k_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
          (v_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
          (o_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
        )
        (mlp): LlamaMLP(
          (gate_proj): WQLinear_GEMM(in_features=4096, out_features=11008, bias=False, w_bit=4, group_size=128)
          (up_proj): WQLinear_GEMM(in_features=4096, out_features=11008, bias=False, w_bit=4, group_size=128)
          (down_proj): WQLinear_GEMM(in_features=11008, out_features=4096, bias=False, w_bit=4, group_size=128)
          

In [ ]:
q_model_awq_memory = 0

In [ ]:
# Evaluate the awq quantized model
q_model_awq_perplexity = evaluate(q_model_awq, tokenizer)
print(f"\nAWQ INT4 model perplexity: {q_model_awq_perplexity:.2f}")
gc.collect()
torch.cuda.empty_cache()

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

evaluating...: 100%|██████████| 100/100 [01:20<00:00,  1.24it/s]



AWQ INT4 model perplexity: 7.03


In [ ]:
q_model_awq_ppl = 0

Structure of AWQ quantized models

In [ ]:
print(q_model_awq.model.decoder.layers[0].self_attn.q_proj.qweight)
print(q_model_awq.model.decoder.layers[0].self_attn.q_proj.qweight.shape)

AttributeError: 'LlamaModel' object has no attribute 'decoder'

In [ ]:
print(q_model_awq.model.decoder.layers[0].self_attn.q_proj.scales)
print(q_model_awq.model.decoder.layers[0].self_attn.q_proj.scales.shape)

In [ ]:
print(q_model_awq.model.decoder.layers[0].self_attn.q_proj.qzeros[0][:10])
print(q_model_awq.model.decoder.layers[0].self_attn.q_proj.qzeros.shape)

### Part1-5 Summary

In [ ]:
print(f"Base model: Llama-2-7b-chat-hf")
print(f"Quantization:       FP32            GPTQ         AWQ         NF4 ")
print(f"dtype:              fp32            INT4         INT4        NF4")
print(f"groupsize:           -              128          128         64")
print(f"memory:             {fp_model_memory}             {q_model_gptq_memory}          {q_model_awq_memory}         {q_model_nf_memory}")
print(f"PPL:               {fp_model_ppl}           {q_model_gptq_ppl}        {q_model_awq_ppl}        {q_model_nf_ppl}")


Base model: facebook/opt-125m
Quantization:       FP32            GPTQ         AWQ         NF4 
dtype:              fp32            INT4         INT4        NF4
groupsize:           -              128          128         64


NameError: name 'fp_model_memory' is not defined